# **CASE: SALÁRIO DE FUNCIONÁRIOS**

---

### Instalação de bibliotecas

In [ ]:
# Para geração de correlograma
install.packages("GGally")
# Para cálculo do VIF
install.packages("car")

### Carregamento de bibliotecas

In [ ]:
library(GGally)
library(car)

### Leitura da base de dados

In [ ]:
dados_salario <- read.table(file   = "Salario_Funcionarios.txt",
                            sep    = "\t",
                            header = TRUE)

### Visualização da base de dados

In [ ]:
View(dados_salario)

### Análise exploratória breve

In [ ]:
# Medidas resumo
summary(dados_salario)

In [ ]:
# Tabelas de frequências
table(dados_salario$DESEMPENHO_PROVA)
prop.table(table(dados_salario$DESEMPENHO_PROVA))

table(dados_salario$FLAG_EXPERIENCIA)
prop.table(table(dados_salario$FLAG_EXPERIENCIA))

In [ ]:
# Definindo ordem dos níveis da variável qualitativa ordinal DESEMPENHO_PROVA
dados_salario$DESEMPENHO_PROVA <- factor(dados_salario$DESEMPENHO_PROVA,
                                         levels = c("Razoavel", "Bom", "Otimo"))

### Análise bivariada: variável resposta vs. variáveis explicativas

In [ ]:
# Comando auxiliar para aumentar resolução do gráfico (recomendado apenas no Colab)
reset_options <- options()
options(repr.plot.width = 15, repr.plot.height = 15)

In [ ]:
# Correlograma
ggpairs(dados_salario[, !names(dados_salario) %in% c("ID_FUNCIONARIO", "DESEMPENHO_PROVA", "FLAG_EXPERIENCIA")],
        title = "Correlograma",
        upper = list(continuous = wrap("cor", size = 5)))

In [ ]:
# Boxplots para variáveis qualitativas
boxplot(dados_salario$SALARIO_DEFINIDO ~ dados_salario$DESEMPENHO_PROVA,
        main = "Boxplot de SALARIO_DEFINIDO por DESEMPENHO_PROVA")
boxplot(dados_salario$SALARIO_DEFINIDO ~ dados_salario$FLAG_EXPERIENCIA,
        main = "Boxplot de SALARIO_DEFINIDO por FLAG_EXPERIENCIA")

In [ ]:
# Resetando parâmetros gráficos (recomendado apenas no Colab)
options(reset_options)

### Modelo de regressão linear múltipla

In [ ]:
# Comando auxiliar para para omitir notação científica nos p-valores e controlar largura dos outputs na tela do Colab
options(scipen = 999, width = 200)

In [ ]:
# Ajuste do modelo 1: inicial
regressao_1 <- lm(SALARIO_DEFINIDO ~
                  PRETENSAO_SALARIAL +
                  DESEMPENHO_PROVA +
                  FLAG_EXPERIENCIA,
                data = dados_salario)

summary(regressao_1)

In [ ]:
# Avaliação de colinearidade no modelo 1
data.frame(VIF = vif(regressao_1))

### Intervalos de confiança

In [ ]:
# Escolha do beta (0 para intercepto, ou 1, 2, ... para os ângulos referentes a cada variável explicativa)
beta = 0

In [ ]:
# Cálculo do intervalo
print("Limite inferior")
as.numeric(regressao_1$coefficients[beta + 1] - 1.96 * coef(summary(regressao_1))[beta + 1, "Std. Error"])
print("Limite superior")
as.numeric(regressao_1$coefficients[beta + 1] + 1.96 * coef(summary(regressao_1))[beta + 1, "Std. Error"])

### Qualidade de ajuste

In [ ]:
# Coeficiente de determinação ajustado (R² ajustado)
summary(regressao_1)$adj.r.squared

In [ ]:
# MAE
sum(abs(regressao_1$residuals))/nrow(dados_salario)

In [ ]:
# MAPE
sum(abs(regressao_1$residuals/dados_salario$SALARIO_DEFINIDO))/nrow(dados_salario)

### Análise de resíduos

In [ ]:
# Histograma dos resíduos
hist(x      = regressao_1$residuals,
     main   = "Histograma dos resíduos",
     col    = "darkturquoise",
     border = "white")

In [ ]:
# QQ plot dos resíduos
qqnorm(y    = regressao_1$residuals,
       main = "QQ-plot dos resíduos",
       col  = "darkturquoise",
       pch  = 19)
qqline(y    = regressao_1$residuals,
       col  = "red")

In [ ]:
# Resíduos vs. valores preditos
plot(x    = regressao_1$fitted.values,
     y    = regressao_1$residuals,
     pch  = 19,
     col  = "darkturquoise",
     xlab = "Valores ajustados",
     ylab = "Resíduos",
     main = "Resíduos vs. valores ajustados")

### Exemplo de uso (aplicação) do modelo

In [ ]:
# Criação de base de dados com um novo funcionário, com pretensão salarial de R$ 3.000, com ótimo desempenho na prova de seleção, e com experiência anterior na área
# Obs.: os nomes das colunas devem ser idênticos aos da base utilizada para construção do modelo
novos_dados <- data.frame(PRETENSAO_SALARIAL = c(300),
                          DESEMPENHO_PROVA   = c("Otimo"),
                          FLAG_EXPERIENCIA   = c("Sim"))

In [ ]:
# Aplicação do modelo
novos_dados$SALARIO_DEFINIDO_PREDITO <- predict(regressao_1, novos_dados)
View(novos_dados)